# Pipeline Data understanding and preparation

## Libraries importeren

In [ ]:
# imported libraries
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## DataFrame inladen

In [ ]:
cols_to_use = [
    '#stm_sap_meldnr',
    # 'stm_mon_nr',
    # 'stm_vl_post',
    # 'stm_sap_meld_ddt',
    # 'stm_sap_meldtekst_lang',
    # 'stm_sap_meldtekst',
    'stm_geo_mld',
    # 'stm_geo_mld_uit_functiepl',
    # 'stm_equipm_nr_mld',
    # 'stm_equipm_soort_mld',
    # 'stm_equipm_omschr_mld',
    # 'stm_km_van_mld',
    # 'stm_km_tot_mld',
    'stm_prioriteit',
    # 'stm_status_melding_sap',
    # 'stm_aanngeb_ddt',
    # 'stm_oh_pg_gst',
    'stm_geo_gst',
    # 'stm_geo_gst_uit_functiepl',
    # 'stm_equipm_nr_gst',
    # 'stm_equipm_soort_gst',
    # 'stm_equipm_omschr_gst',
    # 'stm_km_van_gst',
    # 'stm_km_tot_gst',
    # 'stm_oorz_groep',
    'stm_oorz_code',
    # 'stm_oorz_tkst',
    'stm_fh_ddt',
    # 'stm_fh_status',
    # 'stm_sap_storeind_ddt',
    # 'stm_tao_indicator',
    # 'stm_tao_indicator_vorige',
    # 'stm_tao_soort_mutatie',
    # 'stm_tao_telling_mutatie',
    # 'stm_tao_beinvloedbaar_indicator',
    # 'stm_evb',
    # 'stm_sap_melddatum',
    # 'stm_sap_meldtijd',
    'stm_contractgeb_mld',
    # 'stm_functiepl_mld',
    'stm_techn_mld',
    # 'stm_contractgeb_gst',
    # 'stm_functiepl_gst',
    'stm_techn_gst',
    # 'stm_aanngeb_dd',
    # 'stm_aanngeb_tijd',
    'stm_aanntpl_dd',
    'stm_aanntpl_tijd',
    # 'stm_arbeid',
    # 'stm_progfh_in_datum',
    # 'stm_progfh_in_tijd',
    # 'stm_progfh_in_invoer_dat',
    # 'stm_progfh_in_invoer_tijd',
    # 'stm_progfh_in_duur',
    # 'stm_progfh_gw_tijd',
    # 'stm_progfh_gw_lwd_datum',
    # 'stm_progfh_gw_lwd_tijd',
    # 'stm_progfh_gw_duur',
    # 'stm_progfh_gw_teller',
    # 'stm_afspr_aanvangdd',
    # 'stm_afspr_aanvangtijd',
    'stm_fh_dd',
    'stm_fh_tijd',
    'stm_fh_duur',
    # 'stm_reactie_duur',
    # 'stm_sap_storeinddatum',
    # 'stm_sap_storeindtijd',
    # 'stm_oorz_tekst_kort',
    # 'stm_pplg_van',
    # 'stm_pplg_naar',
    # 'stm_dstrglp_van',
    # 'stm_dstrglp_naar'
]

In [ ]:
df = pd.read_csv("data/sap_storing_data_hu_project.csv", index_col=0, usecols=cols_to_use, engine='pyarrow')
df.head()

In [ ]:
def remove_outlier(df_in, col_name, k=3):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3 - q1 
    fence_low  = q1 - k * iqr
    fence_high = q3 + k * iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

## Prepareren Target variabele

De gekozen target variabele is de tijd (in minuten) vanaf het moment dat de aannemer ter plaatse is tot functie herstel

In [ ]:
# alle relevante kolommen veranderen naar datetimes
df['stm_aanntpl_tijd'] = pd.to_datetime(df['stm_aanntpl_tijd'], format='%H:%M:%S', errors='coerce')
df['stm_aanntpl_dd'] = pd.to_datetime(df['stm_aanntpl_dd'], format='%d/%m/%Y', errors='coerce')
df['stm_fh_ddt'] = pd.to_datetime(df['stm_fh_ddt'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

df[['stm_aanntpl_tijd', 'stm_aanntpl_dd', 'stm_fh_ddt']].info()

In [ ]:
df['stm_aanntpl_tijd'] = df['stm_aanntpl_tijd'].astype('str')
df['stm_aanntpl_dd'] = df['stm_aanntpl_dd'].astype('str')
# de twee kolommen samen voegen om er 1 datetime feature van te maken
df["aanntpl_ddt"] = df["stm_aanntpl_dd"] + " " + df["stm_aanntpl_tijd"].apply(lambda x: x.split(' ')[-1])
df['aanntpl_ddt'] = pd.to_datetime(df['aanntpl_ddt'], format='%Y-%m-%d %H:%M:%S', errors='coerce') 
print(df['aanntpl_ddt'].isna().sum())
df = df.dropna(subset=['aanntpl_ddt'])
df['aanntpl_ddt'].sample(30)

In [ ]:
# bereken de duur van aannemer ter plaatse tot functieherstel in minuten
df['anm_tot_fh'] = df['stm_fh_ddt'] - df['aanntpl_ddt']
df['anm_tot_fh'] = df['anm_tot_fh'].apply(lambda x: x.seconds/60 + x.days * (24*60))
df = df.dropna(subset=['anm_tot_fh'])
df['anm_tot_fh'].sample(30)


In [ ]:
# verwijder alle negatieve waardes, omdat dit er niet zo veel zijn
df = df[df['anm_tot_fh'] >= 0]

In [ ]:
df['anm_tot_fh'].plot.box()

In [ ]:
df = df[(df['anm_tot_fh'] >= 5) & (df['anm_tot_fh'] <= 480)]
len(df)

In [ ]:
df['anm_tot_fh'].plot.box()

# Baseline
Voor de baseline is er gekozen om de gemiddelde te nemen van de target variable.

In [ ]:
import baseline
baseline_mse, baseline_r2 = baseline.calculate_baseline(df)

print('Baseline MSE: ', baseline_mse)
print('Baseline R2: ', baseline_r2)

# Resultaten
Hierboven is te zien dat de baseline een R2 score heeft van 0.0 en de MSE is ongeveer 5876.64. Dit is niet een goede score, maar goed genoeg voor een baseline.